# Spotify Music Data Collection

In [ ]:
import spotipy
import spotipy.util as util
from requests import get
from datetime import datetime
import os
import csv
from apscheduler.schedulers.blocking import BlockingScheduler

def job_function():
    def Spotify_API(username, scope, client_id, client_secret, redirect_uri):
        try:
            token = util.prompt_for_user_token(
                    username=username,
                    scope=scope,
                    client_id=client_id,
                    client_secret=client_secret,
                    redirect_uri=redirect_uri
                    )
        except:
            os.remove(f".cache-{username}")
            token = util.prompt_for_user_token(
                    username=username,
                    scope=scope,
                    client_id=client_id,
                    client_secret=client_secret,
                    redirect_uri=redirect_uri
                    )

        # Create spotify object with authorisation
        spotifyobj = spotipy.Spotify(auth=token)

        # Get current playing track
        currenttrack = spotifyobj.current_user_playing_track()

        # Connvert UNIX time stamp to YMDHMS
        time = currenttrack['timestamp']/1E3
        timedate = datetime.fromtimestamp(int(time)).strftime('%Y-%m-%d %H:%M:%S')
        
        # Track features
        trackplaying = currenttrack['is_playing'] #True/False track is playing
        trackid = currenttrack['item']['uri'] #Spotify track ID
        name = currenttrack['item']['name'] #song name
        artist = currenttrack['item']['album']['artists'][0]['name'] #artist name
        artist2 = currenttrack['item']['album']['artists'][0]['uri']
        artists = spotifyObject.artist(artist2)
        genres = artists['genres'] #genres
        
        # Audio analysis
        analysis = spotifyObject.audio_features(trackid)
        return [timedate, analysis[0]['energy'], analysis[0]['tempo'], analysis[0]['loudness'],
                analysis[0]['danceability'], analysis[0]['acousticness'], analysis[0]['instrumentalness']]


    # Spoitfy Credentials (removed from script for privacy reasons)
    username = '*****'
    client_id = '*****'
    client_secret = '*****'
    redirect_uri = '*****'
    scope = 'user-read-currently-playing'

    spotifydata = Spotify_API(username, scope, client_id, client_secret, redirect_uri)


    if spotifydata[1] == False:
        print('No track playing')
        
        # Input blank data into a csv file
        with open('music.csv', mode='a') as file:
            writer = csv.writer(file)
            writer.writerow()
    else:
        print(spotifydata)
        
        # Input music data into a csv file
        with open('music.csv', mode='a') as file:
            writer = csv.writer(file)
            writer.writerow(spotifydata)


# Schedule job_function to be called every 10 minutes for the week specified:
scheduler = BlockingScheduler()
scheduler.add_job(job_function, 'interval', minutes=10, start_date='2019-12-20 00:00:00', end_date='2019-12-28 00:00:00')
scheduler.start()